# Experiment Results Analysis

This notebook extracts experiment data into the exact DataFrame format requested.

In [1]:
import json
import pandas as pd
from typing import Dict, List, Any

In [2]:
def load_experiment_results(file_path: str) -> Dict[str, Any]:
    """Load experiment results from JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def extract_experiment_dataframe(results: Dict[str, Any]) -> pd.DataFrame:
    """
    Extract experiment data into a DataFrame with the exact columns requested.
    
    Args:
        results: Experiment results dictionary
        
    Returns:
        DataFrame with columns: Rounds, #Agents, Prompt, Model own, Model other agents, 
        Temperature self, Initial Preference, Agreement?, Agreed Principle
    """
    
    # Extract basic experiment info
    general_info = results.get('general_information', {})
    agents = results.get('agents', [])
    
    # Get experiment configuration
    rounds = general_info.get('phase2_rounds', 20)  # Default if not found
    num_agents = len(agents)
    
    # Extract consensus information
    consensus_reached = general_info.get('consensus_reached', False)
    consensus_principle = general_info.get('consensus_principle', None)
    
    # Map principle codes to numbers (based on your example showing "3")
    principle_to_number = {
        'maximizing_floor': 1,
        'maximizing_average': 2, 
        'maximizing_average_with_floor': 3,
        'maximizing_average_with_range': 4
    }
    
    agreed_principle_num = principle_to_number.get(consensus_principle, None) if consensus_principle else None
    
    # Create rows for each agent
    rows = []
    for agent in agents:
        # Get agent's model
        agent_model = agent.get('model', 'Unknown')
        
        # Get other agents' models (excluding current agent)
        other_models = [a.get('model', 'Unknown') for a in agents if a.get('name') != agent.get('name')]
        
        # Get agent's initial preference (first choice)
        initial_preference = None
        if 'phase_1' in agent and 'initial_ranking' in agent['phase_1']:
            ranking_data = agent['phase_1']['initial_ranking'].get('ranking_result', {})
            rankings = ranking_data.get('rankings', [])
            
            # Find rank 1 (first choice)
            for ranking in rankings:
                if ranking.get('rank') == 1:
                    principle = ranking.get('principle', '')
                    initial_preference = principle_to_number.get(principle, None)
                    break
        
        row = {
            'Rounds': rounds,
            '#Agents': num_agents,
            'Prompt': agent.get('personality', ''),
            'Model own': agent_model,
            'Model other agents': other_models,
            'Temperature self': agent.get('temperature', 0),
            'Initial Preference': initial_preference,
            'Agreement?': 1 if consensus_reached else 0,  # 1 for Yes, 0 for No
            'Agreed Principle': agreed_principle_num
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

## Example Usage

In [5]:
# Load the experiment results
results = load_experiment_results("/Users/lucasmuller/Desktop/Githubg/Rawls_v3/hypothesis_2_&_4/config_01_results_20250810_094637.json")

# Extract the DataFrame with exact columns you want
df = extract_experiment_dataframe(results)

print("Extracted DataFrame:")
display(df)

Extracted DataFrame:


,Rounds,#Agents,Prompt,Model own,Model other agents,Temperature self,Initial Preference,Agreement?,Agreed Principle
0,20,5,You are an american college student.,google/gemini-2.5-pro,"[google/gemini-2.5-pro, openai/gpt-4.1-mini, a...",0.0,1,1,1
1,20,5,You are an american college student.,google/gemini-2.5-pro,"[google/gemini-2.5-pro, openai/gpt-4.1-mini, a...",0.0,1,1,1
2,20,5,You are an american college student.,openai/gpt-4.1-mini,"[google/gemini-2.5-pro, google/gemini-2.5-pro,...",0.0,1,1,1
3,20,5,You are an american college student.,anthropic/claude-sonnet-4,"[google/gemini-2.5-pro, google/gemini-2.5-pro,...",0.0,1,1,1
4,20,5,You are an american college student.,anthropic/claude-sonnet-4,"[google/gemini-2.5-pro, google/gemini-2.5-pro,...",0.0,1,1,1


In [ ]:
# You can also save the DataFrame to CSV for further analysis
df.to_csv('logs/experiment_analysis.csv', index=False)
print("DataFrame saved to logs/experiment_analysis.csv")

In [ ]:
# Check the DataFrame structure
print("DataFrame Info:")
print(df.info())
print("\nDataFrame Shape:", df.shape)
print("\nColumn Names:", list(df.columns))